In [1]:
# !pip install transformers
import torch
import pickle
from torch import nn
import numpy as np
import pandas as pd
from transformers import *
from sklearn.metrics import roc_curve, auc
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler, DataLoader, random_split

In [2]:
class InputExample(object):
    def __init__(self, id, text, labels=None):
        self.id = id
        self.text = text
        self.labels = labels

class InputFeatures(object):
    def __init__(self, input_ids, input_mask, segment_ids, label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids

In [3]:
def get_train_examples(train_file):
    train_df = pd.read_csv(train_file)
    ids = train_df['id'].values
    text = train_df['comment_text'].values
    labels = train_df[train_df.columns[2:]].values
    examples = []
    for i in range(len(train_df)):
        examples.append(InputExample(ids[i], text[i], labels=labels[i]))
    return examples

In [4]:
def get_features_from_examples(examples, max_seq_len, tokenizer):
    features = []
    for i,example in enumerate(examples):
        tokens = tokenizer.tokenize(example.text)
        if len(tokens) > max_seq_len - 2:
            tokens = tokens[:(max_seq_len - 2)]
        tokens = ["[CLS]"] + tokens + ["[SEP]"]
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)
        segment_ids = [0] * len(tokens)
        padding = [0] * (max_seq_len - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding
        assert len(input_ids) == max_seq_len
        assert len(input_mask) == max_seq_len
        assert len(segment_ids) == max_seq_len
        label_ids = [float(label) for label in example.labels]
        features.append(InputFeatures(input_ids=input_ids,
                                      input_mask=input_mask,
                                      segment_ids=segment_ids,
                                      label_ids=label_ids))
    return features

In [5]:
def get_dataset_from_features(features):
    input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.float)
    dataset = TensorDataset(input_ids,
                            input_mask,
                            segment_ids,
                            label_ids)
    return dataset

In [29]:
class KimCNN(nn.Module):
    def __init__(self, embed_num, embed_dim, dropout=0.1, kernel_num=3, kernel_sizes=[2,3,4], num_labels=2):
        super().__init__()
        self.num_labels = num_labels
        self.embed_num = embed_num
        self.embed_dim = embed_dim
        self.dropout = dropout
        self.kernel_num = kernel_num
        self.kernel_sizes = kernel_sizes
        self.embed = nn.Embedding(self.embed_num, self.embed_dim)
        self.convs = nn.ModuleList([nn.Conv2d(1, self.kernel_num, (k, self.embed_dim)) for k in self.kernel_sizes])
        self.dropout = nn.Dropout(self.dropout)
        self.classifier = nn.Linear(len(self.kernel_sizes)*self.kernel_num, self.num_labels)
        
    def forward(self, inputs, labels=None):
        output = inputs.unsqueeze(1)
        output = [nn.functional.relu(conv(output)).squeeze(3) for conv in self.convs]
        output = [nn.functional.max_pool1d(i, i.size(2)).squeeze(2) for i in output]
        output = torch.cat(output, 1)
        output = self.dropout(output)
        logits = self.classifier(output)
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1, self.num_labels))
            return loss
        else:
            return logits

In [7]:
device = torch.device(type='cuda')
pretrained_weights = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
basemodel = BertModel.from_pretrained(pretrained_weights)
basemodel.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [8]:
seq_len = 256
train_file = 'train.csv'
train_examples = get_train_examples(train_file)
train_features = get_features_from_examples(train_examples, seq_len, tokenizer)
train_dataset = get_dataset_from_features(train_features)


train_val_split = 0.1
train_size = int(len(train_dataset)*(1-train_val_split))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

batch = 8
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch)
val_sampler = SequentialSampler(val_dataset)
val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=batch)

In [30]:
embed_num = seq_len 
embed_dim = basemodel.config.hidden_size 
dropout = basemodel.config.hidden_dropout_prob
kernel_num = 3
kernel_sizes = [2,3,4]
num_labels = 6

model = KimCNN(embed_num, embed_dim, dropout=dropout, kernel_num=kernel_num, kernel_sizes=kernel_sizes, num_labels=num_labels)
model.to(device)

KimCNN(
  (embed): Embedding(256, 768)
  (convs): ModuleList(
    (0): Conv2d(1, 3, kernel_size=(2, 768), stride=(1, 1))
    (1): Conv2d(1, 3, kernel_size=(3, 768), stride=(1, 1))
    (2): Conv2d(1, 3, kernel_size=(4, 768), stride=(1, 1))
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (classifier): Linear(in_features=9, out_features=6, bias=True)
)

In [34]:
lr = 3e-5
epochs = 1
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

for i in range(epochs):
    print('-----------EPOCH #{}-----------'.format(i+1))
    model.train()
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch
        with torch.no_grad():
            inputs,_ = basemodel(input_ids, segment_ids, input_mask)
        loss = model(inputs, label_ids)
        loss = loss.mean()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()        
    
    y_true = []
    y_pred = []

    model.eval()
    for step, batch in enumerate(val_dataloader):
        batch = tuple(t.to(device) for t in batch)
        val_input_ids, val_input_mask, val_segment_ids, val_label_ids = batch
        with torch.no_grad():
            val_inputs,_ = basemodel(val_input_ids, val_segment_ids, val_input_mask)
            logits = model(val_inputs)
        y_true.append(val_label_ids)
        y_pred.append(logits)

    y_true = torch.cat(y_true, dim=0).float().cpu().detach().numpy()
    y_pred = torch.cat(y_pred, dim=0).float().cpu().detach().numpy()

    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i,label in enumerate(labels):
        fpr[label], tpr[label], _ = roc_curve(y_true[:, i], y_pred[:, i])
        roc_auc[label] = auc(fpr[label], tpr[label])

    print('ROC AUC per label:')
    for label in labels:
        print(label, ': ', roc_auc[label])


batches: 0it [00:00, ?it/s]
batches: 3it [00:00, 26.77it/s]

------------------------------------
             EPOCH #1              
------------------------------------



batches: 6it [00:00, 25.69it/s]
batches: 9it [00:00, 25.02it/s]
batches: 12it [00:00, 24.48it/s]
batches: 15it [00:00, 24.15it/s]
batches: 18it [00:00, 23.90it/s]
batches: 21it [00:00, 23.78it/s]
batches: 24it [00:01, 23.70it/s]
batches: 27it [00:01, 23.65it/s]
batches: 30it [00:01, 23.56it/s]
batches: 33it [00:01, 23.52it/s]
batches: 36it [00:01, 23.59it/s]
batches: 39it [00:01, 23.53it/s]
batches: 42it [00:01, 23.48it/s]
batches: 45it [00:01, 23.41it/s]
batches: 48it [00:02, 23.39it/s]
batches: 51it [00:02, 23.45it/s]
batches: 54it [00:02, 23.46it/s]
batches: 57it [00:02, 23.34it/s]
batches: 60it [00:02, 23.48it/s]
batches: 63it [00:02, 23.42it/s]
batches: 66it [00:02, 23.25it/s]
batches: 69it [00:02, 23.41it/s]
batches: 72it [00:03, 23.37it/s]
batches: 75it [00:03, 23.27it/s]
batches: 78it [00:03, 23.33it/s]
batches: 81it [00:03, 23.42it/s]
batches: 84it [00:03, 23.22it/s]
batches: 87it [00:03, 23.56it/s]
batches: 90it [00:03, 23.47it/s]
batches: 93it [00:03, 23.46it/s]
batches: 96

batches: 672it [00:28, 23.33it/s]
batches: 675it [00:28, 23.20it/s]
batches: 678it [00:29, 23.23it/s]
batches: 681it [00:29, 23.38it/s]
batches: 684it [00:29, 23.28it/s]
batches: 687it [00:29, 23.35it/s]
batches: 690it [00:29, 23.44it/s]
batches: 693it [00:29, 23.35it/s]
batches: 696it [00:29, 23.27it/s]
batches: 699it [00:29, 23.23it/s]
batches: 702it [00:30, 23.18it/s]
batches: 705it [00:30, 23.27it/s]
batches: 708it [00:30, 23.35it/s]
batches: 711it [00:30, 23.36it/s]
batches: 714it [00:30, 23.40it/s]
batches: 717it [00:30, 23.02it/s]
batches: 720it [00:30, 23.38it/s]
batches: 723it [00:30, 23.34it/s]
batches: 726it [00:31, 23.36it/s]
batches: 729it [00:31, 23.38it/s]
batches: 732it [00:31, 23.39it/s]
batches: 735it [00:31, 23.28it/s]
batches: 738it [00:31, 23.32it/s]
batches: 741it [00:31, 23.31it/s]
batches: 744it [00:31, 23.32it/s]
batches: 747it [00:31, 23.37it/s]
batches: 750it [00:32, 23.38it/s]
batches: 753it [00:32, 23.34it/s]
batches: 756it [00:32, 23.45it/s]
batches: 759it

batches: 1326it [00:56, 23.39it/s]
batches: 1329it [00:56, 23.30it/s]
batches: 1332it [00:57, 23.22it/s]
batches: 1335it [00:57, 23.24it/s]
batches: 1338it [00:57, 23.32it/s]
batches: 1341it [00:57, 23.26it/s]
batches: 1344it [00:57, 23.22it/s]
batches: 1347it [00:57, 23.19it/s]
batches: 1350it [00:57, 23.23it/s]
batches: 1353it [00:57, 23.22it/s]
batches: 1356it [00:58, 23.15it/s]
batches: 1359it [00:58, 23.21it/s]
batches: 1362it [00:58, 23.23it/s]
batches: 1365it [00:58, 23.24it/s]
batches: 1368it [00:58, 23.28it/s]
batches: 1371it [00:58, 23.31it/s]
batches: 1374it [00:58, 23.30it/s]
batches: 1377it [00:59, 23.16it/s]
batches: 1380it [00:59, 23.37it/s]
batches: 1383it [00:59, 23.37it/s]
batches: 1386it [00:59, 23.30it/s]
batches: 1389it [00:59, 23.31it/s]
batches: 1392it [00:59, 23.23it/s]
batches: 1395it [00:59, 23.38it/s]
batches: 1398it [00:59, 23.33it/s]
batches: 1401it [01:00, 23.35it/s]
batches: 1404it [01:00, 23.45it/s]
batches: 1407it [01:00, 23.38it/s]
batches: 1410it [01:

...

batches: 17907it [12:49, 23.37it/s]
batches: 17910it [12:49, 23.23it/s]
batches: 17913it [12:49, 23.32it/s]
batches: 17916it [12:49, 23.31it/s]
batches: 17919it [12:49, 23.36it/s]
batches: 17922it [12:50, 23.21it/s]
batches: 17925it [12:50, 23.31it/s]
batches: 17928it [12:50, 23.34it/s]
batches: 17931it [12:50, 23.29it/s]
batches: 17934it [12:50, 23.26it/s]
batches: 17937it [12:50, 23.28it/s]
batches: 17940it [12:50, 23.28it/s]
batches: 17943it [12:51, 23.32it/s]
batches: 17946it [12:51, 23.34it/s]
batches: 17949it [12:51, 23.42it/s]
batches: 17952it [12:51, 23.27it/s]

validation: 0it [00:00, ?it/s]
validation: 4it [00:00, 28.05it/s]
validation: 7it [00:00, 26.64it/s]
validation: 10it [00:00, 25.69it/s]
validation: 13it [00:00, 24.95it/s]
validation: 16it [00:00, 24.57it/s]
validation: 19it [00:00, 24.07it/s]
validation: 22it [00:00, 23.89it/s]
validation: 25it [00:01, 23.64it/s]
validation: 28it [00:01, 23.75it/s]
validation: 31it [00:01, 23.52it/s]
validation: 34it [00:01, 23.63it/s

validation: 571it [00:24, 23.52it/s]
validation: 574it [00:24, 23.59it/s]
validation: 577it [00:24, 23.55it/s]
validation: 580it [00:24, 23.58it/s]
validation: 583it [00:24, 23.48it/s]
validation: 586it [00:24, 23.51it/s]
validation: 589it [00:24, 23.58it/s]
validation: 592it [00:25, 23.53it/s]
validation: 595it [00:25, 23.55it/s]
validation: 598it [00:25, 23.57it/s]
validation: 601it [00:25, 23.52it/s]
validation: 604it [00:25, 23.50it/s]
validation: 607it [00:25, 23.26it/s]
validation: 610it [00:25, 23.36it/s]
validation: 613it [00:26, 23.30it/s]
validation: 616it [00:26, 23.47it/s]
validation: 619it [00:26, 23.49it/s]
validation: 622it [00:26, 23.53it/s]
validation: 625it [00:26, 23.45it/s]
validation: 628it [00:26, 23.41it/s]
validation: 631it [00:26, 23.63it/s]
validation: 634it [00:26, 23.56it/s]
validation: 637it [00:27, 23.60it/s]
validation: 640it [00:27, 23.63it/s]
validation: 643it [00:27, 23.59it/s]
validation: 646it [00:27, 23.56it/s]
validation: 649it [00:27, 23.64it/s]
v

validation: 1180it [00:50, 23.61it/s]
validation: 1183it [00:50, 23.67it/s]
validation: 1186it [00:50, 23.56it/s]
validation: 1189it [00:50, 23.57it/s]
validation: 1192it [00:50, 23.61it/s]
validation: 1195it [00:50, 23.53it/s]
validation: 1198it [00:50, 23.58it/s]
validation: 1201it [00:51, 23.57it/s]
validation: 1204it [00:51, 23.43it/s]
validation: 1207it [00:51, 23.56it/s]
validation: 1210it [00:51, 23.60it/s]
validation: 1213it [00:51, 23.54it/s]
validation: 1216it [00:51, 23.60it/s]
validation: 1219it [00:51, 23.60it/s]
validation: 1222it [00:51, 23.55it/s]
validation: 1225it [00:52, 23.53it/s]
validation: 1228it [00:52, 23.59it/s]
validation: 1231it [00:52, 23.58it/s]
validation: 1234it [00:52, 23.55it/s]
validation: 1237it [00:52, 23.61it/s]
validation: 1240it [00:52, 23.58it/s]
validation: 1243it [00:52, 23.55it/s]
validation: 1246it [00:52, 23.67it/s]
validation: 1249it [00:53, 23.62it/s]
validation: 1252it [00:53, 23.62it/s]
validation: 1255it [00:53, 23.65it/s]
validation: 

validation: 1777it [01:15, 23.55it/s]
validation: 1780it [01:15, 23.56it/s]
validation: 1783it [01:15, 23.52it/s]
validation: 1786it [01:15, 23.52it/s]
validation: 1789it [01:15, 23.50it/s]
validation: 1792it [01:16, 23.50it/s]
validation: 1795it [01:16, 23.58it/s]
validation: 1798it [01:16, 23.59it/s]
validation: 1801it [01:16, 23.48it/s]
validation: 1804it [01:16, 23.47it/s]
validation: 1807it [01:16, 23.50it/s]
validation: 1810it [01:16, 23.52it/s]
validation: 1813it [01:16, 23.54it/s]
validation: 1816it [01:17, 23.66it/s]
validation: 1819it [01:17, 23.58it/s]
validation: 1822it [01:17, 23.50it/s]
validation: 1825it [01:17, 23.50it/s]
validation: 1828it [01:17, 23.52it/s]
validation: 1831it [01:17, 23.61it/s]
validation: 1834it [01:17, 23.56it/s]
validation: 1837it [01:17, 23.59it/s]
validation: 1840it [01:18, 23.58it/s]
validation: 1843it [01:18, 23.66it/s]
validation: 1846it [01:18, 23.57it/s]
validation: 1849it [01:18, 23.52it/s]
validation: 1852it [01:18, 23.55it/s]
validation: 

ROC AUC per label:
toxic :  0.9803054338255165
severe_toxic :  0.9821638109720223
obscene :  0.982796207023597
threat :  0.940847351702358
insult :  0.9779683001676738
identity_hate :  0.9350207637728106


In [56]:
def sigmoid(z):
    s = 1.0 / (1.0 + np.exp(-1.0 * z))
    return s

for i in np.random.randint(0, len(y_pred), size=10):
    string = tokenizer.decode(val_dataset[i][0], skip_special_tokens=True)
    print('---------------------------------')
    print('Comment:')
    print(string)
    preds = dict(zip(labels, sigmoid(y_pred[i])))
    for label in preds:
        print('Prediction:')
        print(label, ': ', preds[label])

---------------------------------
Comment:
I'm aware of the content of the revdel'd edit, although I don't necessarily share your conclusions about it. But I don't ( and didn't ) doubt your diligence at all. The comment Worm remembers Malleus as making about admin account access was indeed a little bizarre, but it's a funny old world. ( talk )
Prediction:
toxic :  0.0030493669
Prediction:
severe_toxic :  0.00017798663
Prediction:
obscene :  0.0015997774
Prediction:
threat :  6.7240784e-05
Prediction:
insult :  0.0010458541
Prediction:
identity_hate :  0.00031439873
---------------------------------
Comment:
The game is also available as part of the Star Wars : Best of PC Collection, which, though it's no longer in production, is still available. My wife bought it for me just a few months ago on Amazon. com.
Prediction:
toxic :  0.0022945297
Prediction:
severe_toxic :  8.801558e-05
Prediction:
obscene :  0.0012358881
Prediction:
threat :  5.452585e-05
Prediction:
insult :  0.00069147
Pr